# FP-growth 算法简介
一种非常好的发现频繁项集算法。
基于Apriori算法构建,但是数据结构不同，使用叫做 FP树 的数据结构结构来存储集合。
### FP-growth 算法步骤
基于数据构建FP树
从FP树种挖掘频繁项集

### FP树

In [ ]:
#FP树的节点结构如下:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue     # 节点名称
        self.count = numOccur     # 节点出现次数
        self.nodeLink = None      # 不同项集的相同项通过nodeLink连接在一起
        # needs to be updated
        self.parent = parentNode  # 指向父节点
        self.children = {}        # 存储叶子节点

### FP-growth 原理
（1）基于数据构建FP树
步骤1:
    遍历所有的数据集合，计算所有项的支持度。
    丢弃非频繁的项。
    基于支持度降序排序所有的项。 
    所有数据集合按照得到的顺序重新整理。
    重新整理完成后，丢弃每个集合末尾非频繁项。 
步骤2: 
    读取每个集合插入FP树中，同时用一个头部链表数据结构维护不同集合的相同项
    最终得到一棵FP树(带头指针表的FP树)

（2）从FP树中挖掘出频繁项集
步骤3:
    对头部链表进行降序排序
    对头部链表节点从小到大遍历，得到条件模式基（conditional pattern base），同时获得一个频繁项集。 条件模式基的值取决于末尾节点，一个频繁项集的支持度由支持度最小的项决定。所以 t 节点的条件模式基的值可以理解为对于以 t 节点为末尾的前缀路径出现次数。
    条件模式基继续构造条件 FP树， 得到频繁项集，和之前的频繁项组合起来，这是一个递归遍历头部链表生成FP树的过程，递归截止条件是生成的FP树的头部链表为空。

#### FP-growth 算法优缺点:
* 优点： 1. 因为 FP-growth 算法只需要对数据集遍历两次，所以速度更快。
        2. FP树将集合按照支持度降序排序，不同路径如果有相同前缀路径共用存储空间，使得数据得到了压缩。
        3. 不需要生成候选集。
        4. 比Apriori更快。
* 缺点： 1. FP-Tree第二次遍历会存储很多中间过程的值，会占用很多内存。
        2. 构建FP-Tree是比较昂贵的。
* 适用数据类型：标称型数据(离散型数据)。

### （1）基于数据构建FP树

main 方法大致步骤:
if __name__ == "__main__":
    simpDat = loadSimpDat()                       #加载数据集。
    initSet = createInitSet(simpDat)              #对数据集进行整理，相同集合进行合并。
    myFPtree, myHeaderTab = createTree(initSet, 3)#创建FP树。
    freqItemList = []
    mineTree(myFPtree, myHeaderTab, 3, set([]), freqItemList) #递归的从FP树中挖掘出频繁项集。
    print freqItemList

In [ ]:
#FP树的节点结构如下:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        '''创建节点时
           input：  nameValue        节点名称
                    numOccur         节点出现次数         
                    parentNode       父节点
        '''
        self.name = nameValue                                                     # 节点名称
        self.count = numOccur                                                     # 节点出现次数
        self.nodeLink = None                                                      # 不同项集的相同项通过nodeLink连接在一起
        # needs to be updated
        self.parent = parentNode  # 指向父节点
        self.children = {}        # 存储叶子节点
    def inc(self, numOccur):
        """inc(对count变量增加给定值)
        """
        self.count += numOccur

    def disp(self, ind=1):
        """disp(用于将树以文本形式显示)
        """
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

- 遍历所有的数据集合，计算所有项的支持度。
- 丢弃非频繁的项。
- 基于支持度降序排序所有的项。 
- 所有数据集合按照得到的顺序重新整理。
- 重新整理完成后，丢弃每个集合末尾非频繁项。 

In [ ]:
def createTree(dataSet, minSup=1):                                            # create FP-tree from dataset but don't mine
    '''input:  dataSet        数据集
               minSup         项最小出现次数
       output：retTree        FP树
               headerTable    头指针表
    '''
    headerTable = {}                                                          # 初始化项的计数字典
                                                                              # go over dataSet twice
                                                                              # first pass counts frequency of occurance  第一次遍历数据集，统计每个元素项的频数
    for trans in dataSet:                                                     # 遍历数据集每一条记录     
        for item in trans:                                                    # 遍历每一项
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]     # 计算频数 ，get() 方法返回指定键的值，如果值不在字典中返回默认值。
    headerTableCopy = headerTable.copy()                                      # 保存项计数字典的副本
    for k in headerTableCopy.keys():                                          # remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())                                     # 创建频繁项集合                                  
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None                               # if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]                               # reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None)                                   # treeNode创建根节点
    
                                                                              # go through dataset 2nd time 
    for tranSet, count in dataSet.items():                                    # items() 函数以列表返回可遍历的(键, 值) 元组数组
        localD = {}
        for item in tranSet:                                                  # put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)             # populate tree with ordered freq itemset
    return retTree, headerTable                                               # return tree and header table

In [ ]:
#读取每个集合插入FP树中，同时用一个头部链表数据结构维护不同集合的相同项，最终得到一棵FP树(带头指针表的FP树)
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:                                           # check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count)                                  # incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None:                                  # update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:                                                        # call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [ ]:
#简单的数据集以及数据包装器
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [ ]:
simpDat = loadSimpDat()
simpDat

In [ ]:
initSet = createInitSet(simpDat)
initSet

In [ ]:
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree

In [ ]:
myFPtree.disp() #使用display()方法给出输的文本表示效果

### （2）从FP树中挖掘出频繁项集

In [ ]:
def ascendTree(leafNode, prefixPath):                                        # ascends from leaf node to root 上溯，收集所有遇到的元素项名称
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [ ]:
def findPrefixPath(basePat, treeNode):                                       # treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [ ]:
findPrefixPath('x', myHeaderTab['x'][1])

In [ ]:
findPrefixPath('z', myHeaderTab['z'][1])

In [ ]:
findPrefixPath('r', myHeaderTab['r'][1])

### 创建条件FP树

In [ ]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[0])]            # (sort header table)
    for basePat in bigL:                                                              # start from bottom of header table
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print 'finalFrequent Item: ',newFreqSet    #append to set
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        #print 'condPattBases :',basePat, condPattBases
        #2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
        #print 'head from conditional tree: ', myHead
        if myHead != None: #3. mine cond. FP-tree
            print('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [ ]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)